# Data Repository Service (DRS) URI Access Examination


In this notebook, we get all the DRS URIs from the data table and check if we can access the file each DRS URI points to. 

This notebook will:
1. Grab all DRS URIs from the data table
2. Resolve each DRS URI and check our access to the file
3. Create a report with all the DRS URIs and our access level to each of them

This Notebook will run successfully with the default Terra Cloud Environment with 4 CPUs.
For improved performance when testing a substantial number of DRS URIs in a workspace (> 10,000) increasing the number of Cloud Environment CPUs to 16 is recommended.

## Background


The Data Repository Service (DRS) API is a standardized set of access methods that are agnostic to cloud infrastructure. Developed by the Global Alliance for Genomics and Health (GA4GH), DRS enable researchers to access data regardless of the underlying architecture of the repository (i.e. Google Cloud, Azure, AWS, etc.) in which it is stored. Terra supports accessing data using the GA4GH standard Data Repository Service (DRS). To learn more look at this link: https://support.terra.bio/hc/en-us/articles/360039330211-Data-Access-with-the-GA4GH-Data-Repository-Service-DRS- 

## Setup 

To run code from the terra-notebook-utils, tabulate, and tqdm library, we first have to install the packages and restart the kernel. 

1. Run the cell below
2. Go to the kernel tab in the menu at the top, and select restart
3. Press the restart in the pop-up menu

In [31]:
# Installing terra-notebook-utils, tabulate, and tqdm library
!pip install --upgrade --no-cache-dir git+https://github.com/DataBiosphere/terra-notebook-utils
!pip install tabulate
!pip install tqdm -U

# Configuring notebook log level to log if the DRS URI has access
import logging
%config Application.log_level = "INFO"  
log = logging.getLogger()

  Cloning https://github.com/DataBiosphere/terra-notebook-utils to /tmp/pip-req-build-nyr2bvkg
  Running command git clone -q https://github.com/DataBiosphere/terra-notebook-utils /tmp/pip-req-build-nyr2bvkg
     |████████████████████████████████| 88 kB 6.0 MB/s eta 0:00:011
     |████████████████████████████████| 53 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 18.9 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 785 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 16.4 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 1

## Set DRS URI Column Location 

The DRS URIs' columns location varies per data table varies. The default for this notebook is BioData Catalyst data. Base on your data can choose from biodata-catalyst, anvil, and tcga. If your data is not from one of these three categories, please type the column name of where the DRS URIs are stored.

In [1]:
# Set variable to either "biodata-catalyst", "anvil", "tcga", or "<INSERT DRS URIs' column name>"
columns_name = "tcga"

## Set Head Variable

The head variable is an optional variable that allows you to run the notebook on the first 10 DRS URIs instead of the all the DRS URIs in the data table.

In [24]:
# Change to True if you want to runs the notebook on the first 10 DRS uri in the data table 
head = False

# Grab all DRS URIs from the data table

The DRS URI is stored under the object_id column in a table.

Using the firecloud library, we are going to search for the object_id column in each table in the data table. If a table contains an object_id column, we check if the content is a string and has 'drs://' at its beginning.

In [3]:
# set imports and environment variables
import json
import os
import requests
import sys
import threading
from pytz import timezone
from datetime import datetime
from firecloud import api as fapi
from subprocess import Popen, PIPE
from tabulate import tabulate
from terra_notebook_utils import drs, table
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import pandas as pd


# Getting the namespace/project and name of the workspace
namespace = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']

In [36]:
# Assigning column search key (Add workspace option)  #TODO: Ask micheal about "ga4gh_drs_uri"
data_column_map = {"biodata-catalyst":["object_id"], "anvil":["object_id"]}

# if dataset is "tcga" add list of unique cols - want to search for DRS uris in all cols in all tables
if columns_name == "tcga":
    tcga_cols = set()
    for e in data_tables_list:
        key_list = list(e["attributes"].keys())
        tcga_cols.update([key for key in key_list])
    data_column_map["tcga"] = list(tcga_cols)
    

# get each table's entities with attributes and type 
data_tables_list = fapi.get_entities_with_type(namespace, workspace).json()

# Creating the master DRS URIs dictionary 
DRS_uris_dict = {}

# Going through each item in response (aka one row/entity in all rows from all tables)
for table in data_tables_list:
    table_columns = table["attributes"]  # tuples (col_name, val)
    # if columnns for an entity in tcga dict list, add row/entity to dictionary
    object_id_column = dict(filter(lambda column: column[0] in data_column_map[column_name_search_key], table_columns.items()))
    
    # Checking if the object_id column's contents is a string and start with 'drs://' 
    if object_id_column:
        for col, val in object_id_column.items():
            if type(val) is str and val.startswith('drs://'):
                DRS_uris_dict[val] = {'table_name': table['entityType'], 
                                       'row_id' : table['name'], 
                                       'drs_uri' : val}

                # Outputting the amount of DRS URIs found
print(f"Found {len(DRS_uris_dict)} DRS uris in this workspace")

# If head is true, runs the notebook on the first 10 DRS URI in the data table            
if head:
    for i in range(len(DRS_uris_dict)-1):
        DRS_uris_dict.popitem()

Found 3406 DRS uris in this workspace


# Resolve DRS URIs and Checking File Access

DRS creates a unique ID mapping that allows for flexible file retrieval. The unique mapping is the DRS Uniform Resource Identifier (URI) - a string of characters that uniquely identifies a particular cloud-based resource (similar to URIs) and is agnostic to the cloud infrastructure where it physically exists. To learn where the file physically exists on the cloud, we must resolve the DRS through a backend service called Martha that will unmap the DRS URI to get the file's google bucket file path.

In this step, we check if all the DRS URIs can be resolved and check our access to the file it points to in the google bucket. If the DRS URI can't be resolved, we will record the error that will, later on, be shown in the report in the last step.

One of the most common reasons the DRS URIs can not resolve is "Fence is not linked". If you have this error, it is probably because the data is controlled-access. To use controlled-access data on Terra, you will need to link your Terra user ID to your authorization account (such as a dbGaP account). Linking to external servers will allow Terra to automatically determine if you can access controlled datasets hosted in Terra (ex. TCGA, TOPMed, etc.) based on your valid dbGaP applications. Go to this link to learn more: https://support.terra.bio/hc/en-us/articles/360038086332

## Creating Resolve DRS URI Function 

The `resolve_drs_uri` fuction will resolve and check file access for a given DRS URI


In [29]:
def resolve_drs_uri(drs_url):
    try:
        # Trying to check access to file by reading the first 10 bytes
        drs.head(drs_url)
        
        # Logging if the call succeed
        log.info(f"SUCCESS  : {drs_url}")
        
        # Saving error to the DRS URI information dictionary
        DRS_uris_dict[drs_url]['drs_unresolved_error'] = "None"
    except drs.GSBlobInaccessible:
        # Logging if the call failed
        log.warning(f"NO ACCESS: {drs_url}")
        # Saving the error if failed
        error = sys.exc_info()[1]
        if " Error:" in error:
            error = str((sys.exc_info()[1])).split(" Error:")[1] 
            if "ProviderUser" in error:
                error = error.split("ProviderUser")[0]
        
        # Outputting the error to console
        print(error)
        
        # Outputting the extra error info for the "Fence account not linked" error to console
        if "Fence account not linked" in error:
            print('''One of the most common reasons the DRS URIs can not resolve is "Fence is not linked". 
If you have this error, it is probably because the data is controlled-access. 
To use controlled-access data on Terra, you will need to link your Terra user ID to your authorization account (such as a dbGaP account). 
Linking to external servers will allow Terra to automatically determine if you can access controlled datasets hosted in Terra (ex. TCGA, TOPMed, etc.) based on your valid dbGaP applications. 
Go to this link to learn more: https://support.terra.bio/hc/en-us/articles/360038086332''')
        
        # Saving error to the DRS URI information dictionary
        DRS_uris_dict[drs_url]['drs_unresolved_error'] = error
    
    # Returing the DRS URI information dictionary 
    return {drs_url : DRS_uris_dict[drs_url]}


## Running the Resolve DRS URI Function

Running all the DRS URI found through the resolve_drs_uri fuction in parallel groups

In [32]:
# Outputting the start of resolving DRS URIs
print("Resolving "+ str(len(DRS_uris_dict)) + " DRS uris")

# Assigning the count variable to keep track of resolved DRS URIs
resolved_DRS_uris_count = 0

# Suppress DRS resolution info messages
drs.logger.disabled = True 

# Setting the list of Threads
NUMBER_OF_PROCESSES = 20
THREADS_PER_PROCESS = 10

# Creating batches of Threads
def check_batch(DRS_uris_dict_keys):
    # Creating temp DRS URI information dictionary 
    temp_DRS_uris_dict = {}
    
    # Creating threads for each DRS URIs
    with ThreadPoolExecutor() as e:
        for drs_uri, temp_dict in zip(DRS_uris_dict_keys, e.map(resolve_drs_uri, DRS_uris_dict_keys)):
            # Updating DRS URI information dictionary 
            temp_DRS_uris_dict.update(temp_dict)
    
    # Returing the DRS URI information dictionary 
    return temp_DRS_uris_dict


# Creating number of batches of Threads
batches = [list(DRS_uris_dict.keys())[i:i+THREADS_PER_PROCESS]
            for i in range(0, len(DRS_uris_dict.keys()), THREADS_PER_PROCESS)]

# Creating Pool of batches
with ProcessPoolExecutor(max_workers=NUMBER_OF_PROCESSES) as e:
    for number, prime in tqdm(zip(batches, e.map(check_batch, batches)), total=len(batches)):
        # Updating DRS URI information dictionary 
        DRS_uris_dict.update(prime)


Resolving 2 DRS uris


  0%|          | 0/1 [00:00<?, ?it/s]

2021-05-05 07:46:08::WARNING  NO ACCESS: drs://dg.4DFC:35520334-9761-4d4a-8852-12778565dfa1
2021-05-05 07:46:09::WARNING  NO ACCESS: drs://dg.4DFC:dd86334f-95e7-4949-94d0-90e211ccdfa8


TypeError: argument of type 'GSBlobInaccessible' is not iterable

# Create a report

In this report, we will first print out statistics of the DRS URI data. The information record in the statistics is DRS URIs that were found in the workspace, that were resolved, that were unresolved. Also, files with and without access. Lastly, errors from DRS URIs that are not resolved or files without access.

The second part of the report is the table of DRS URI data that includes the columns: table_name, row_id, drs_uri, drs_unresolved_error, file_access_error, file_path, is_resolved, and file_access 



In [ ]:
# Checking if there is unresolved_DRS_URIs_errors to report
unresolved_DRS_uris_errors_list = []
for value in DRS_uris_dict.values():
    if value['drs_unresolved_error'] not in 'None':
        unresolved_DRS_uris_errors_list.append(value['drs_unresolved_error'])
    
if unresolved_DRS_uris_errors_list:
    unresolved_DRS_uris_errors = ", ".join(set(unresolved_DRS_uris_errors_list))
    if any("Fence account not linked" in errors for errors in unresolved_DRS_uris_errors_list):
        unresolved_DRS_uris_errors =  unresolved_DRS_uris_errors + ''' ~ One of the most common reasons the DRS URIs can not resolve is "Fence is not linked". 
If you have this error, it is probably because the data is controlled-access. 
To use controlled-access data on Terra, you will need to link your Terra user ID to your authorization account (such as a dbGaP account). 
Linking to external servers will allow Terra to automatically determine if you can access controlled datasets hosted in Terra (ex. TCGA, TOPMed, etc.) based on your valid dbGaP applications. 
Go to this link to learn more: https://support.terra.bio/hc/en-us/articles/360038086332'''
else:
    unresolved_DRS_uris_errors = "None"
    
    
# Setting the title for the report
title = "Data Repository Service (DRS) URI Access Examination Report"

# Creating the statistics for the report
stats = f'''

DRS URIs found in the workspace: {len(DRS_uris_dict)} \n
        
DRS URIs resolved: {len(DRS_uris_dict)-len(unresolved_DRS_uris_errors_list)}
 
DRS URIs not resolved: {len(unresolved_DRS_uris_errors_list)}
        
Errors found from DRS URIs that are not resolved: {len(unresolved_DRS_uris_errors_list)}
        
Distinct errors from DRS URIs that are not resolved: {unresolved_DRS_uris_errors}
'''

# Outputting statistics
print(f'''
_______________________________________________________________________

:: {title} ::
_______________________________________________________________________

{stats}''')



# Outputting the table of the master DRS URIs dictionary
pd.DataFrame.from_dict(DRS_uris_dict).transpose()


## (Optional) Save results to the Google Bucket

To save the results in a HTML file in the google bucket, please run the cell below.

In [ ]:
# Converting Report to HTML format 
html_report = f'''<html lang="en">
<head>
<title>{title}</title>''' + '''
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
* {
  box-sizing: border-box;
}

body {
  font-family: Arial, Helvetica, sans-serif;
}

/* Style the header */
header {
  background-color: #73af42;
  padding: 30px;
  text-align: center;
  font-size: 35px;
  color: white;
}

/* Create boxes that floats next to each other */
article {
  float: left;
  padding: 20px;
  width: 100%;
  background-color: #f7f7f7;
  height: 300px; /* only for demonstration, should be removed */
}

/* Style the list inside the menu */
article ul {
  list-style-type: none;
  padding: 0;
}

/* Clear floats after the columns */
section:after {
  content: "";
  display: table;
  clear: both;
}

/* Style the footer */
footer {
  background-color: white;
  padding: 10px;
  text-align: center;
  color: white;
}

/* Responsive layout - makes the two columns/boxes stack on top of each other instead of next to each other, on small screens */
@media (max-width: 600px) {
  nav, article {
    width: 100%;
    height: auto;
  }
}
</style>
</head>
<body>
''' + f'''
<header>
  <h2>{title}</h2>
</header>
<section>
''' + f'''
  <article>
    <h2>
        <ul>
          <li>DRS URIs found in the workspace: {len(DRS_uris_dict)}</li>
          <li>DRS URIs resolved: {len(DRS_uris_dict)-len(unresolved_DRS_uris_errors_list)}</li>
          <li>DRS URIs not resolved: {len(unresolved_DRS_uris_errors_list)}</li>
          <li>Errors found from DRS URIs that are not resolved: {len(unresolved_DRS_uris_errors_list)}</li>
          <li>Distinct errors from DRS URIs that are not resolved: {unresolved_DRS_uris_errors}</li>
        </ul>
      </h2>
  </article>
</section>
<footer>
  <p>{tabulate(pd.DataFrame.from_dict(DRS_uris_dict).transpose(), headers='keys', tablefmt="html")}</p>
</footer>

</body>
</html>
'''

# Getting Timezone
tz = timezone('EST')

# Getting Today Date and Time
today = datetime.now(tz).strftime('%Y_%m_%d_%H_%M_%S')

# HTML File Name
html_file_name = f"DRS_URI_Access_Examination_{today}.html"

# Creating the DRS_URI_Access_Examination.html file
with open(html_file_name, "w") as f:
    f.write(html_report)

# Getting the bucket path of the workspace
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

# Getting the bucket id of the workspace
bucket_id = str(WORKSPACE_BUCKET).replace("gs://", "")

# Copying the DRS_URI_Access_Examination.html file to the google bucket
!gsutil cp {html_file_name} {WORKSPACE_BUCKET}/ 2>&1

# Printing path to the report file in the google bucket
print("\n TO View Result go to https://console.cloud.google.com/storage/browser/_details/" + bucket_id + "/" + html_file_name)
